## Config

Prepping things up for TPU

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
VERSION = "nightly"  #@param ["1.5" , "20200516", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  59222      0 --:--:-- --:--:-- --:--:-- 59222
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-nightly ...
Uninstalling torch-1.5.1+cu101:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][106.7 MiB/106.7 MiB]                                                
Operation completed over 1 objects/106.7 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly-cp36-cp36m-linux_x86_64.whl...
- [1 files][122.0 MiB/122.0 MiB]                                                
Operation completed over 1 objects/122.0 MiB.         

In [3]:
import os
from pathlib import Path
from google.colab import drive
import shutil

def create_path(path):
    if not os.path.isdir(path):
        path.mkdir(parents=True, exist_ok=True)
    return path

colab_path = Path('/content')

In [4]:
create_path(colab_path/'dataset');
create_path(colab_path/'models');

!git clone --quiet 'https://github.com/tezike/download_google_drive.git'
os.chdir('download_google_drive')
!python download_gdrive.py '10rH0nAxa7mWS289xIyRP-mOOowqiIolL' '../dataset/temp.tgz'
shutil.rmtree('../download_google_drive')
os.chdir('..')

## Colab_setup

In [5]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = Path('/content/drive/My Drive')
base_path = create_path(root_dir/'Rakuten')
base_path

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


PosixPath('/content/drive/My Drive/Rakuten')

## Imports

In [6]:
!pip install transformers -q

     |████████████████████████████████| 675kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 15.4MB/s 
     |████████████████████████████████| 3.8MB 23.4MB/s 
     |████████████████████████████████| 890kB 33.6MB/s 


In [7]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

from tqdm import notebook
from pathlib import Path

from torch.nn.utils.rnn import pack_padded_sequence

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

prepping things up for TPU

In [8]:
import torch_xla
import torch_xla.core.xla_model as xm #handles most of the basic tasks
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl #handles dataloading on multiple processes
import torch_xla.distributed.xla_multiprocessing as xmp

## Config

In [28]:
class Config():
    def __init__(self):
        self.MODEL_NAME = 'camembert-base'
        # self.LM_MODEL = transformers.CambertForMaskedLM.from_pretrained(self.MODEL_NAME)
        self.CLAS_MODEL = transformers.CamembertModel #.from_pretrained(MODEL_NAME)
        self.TOKENIZER = transformers.CamembertTokenizer.from_pretrained(
                    pretrained_model_name_or_path=self.MODEL_NAME,
                    do_lower_case=True,
                    )
        self.MODEL_CONFIG = transformers.CamembertConfig.from_pretrained(self.MODEL_NAME)
        self.COLAB_PATH = Path('/content')
        self.BASE_PATH = base_path
        self.DATA_PATH = create_path(base_path/'dataset')
        self.MODEL_PATH = create_path(base_path/'models')
        self.TEST_FILE = self.COLAB_PATH/'SIGIR-2020-EComDC-release/data/x_test_task1_phase1.tsv'
        self.MAX_SEQ_LEN = 256
        self.DEVICE = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
        self.TRAIN_BATCH_SIZE = 32
        self.VALID_BATCH_SIZE = 16
        self.NUM_EPOCHS = 

config = Config()

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


## DATASET

In [10]:
class BertDataset(torch.utils.data.Dataset):
    def __init__(self, text, label):
        self.text, self.label = text, label
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_SEQ_LEN

    def __len__(self):
        return len(self.label)

    def __getitem__(self, i):
        # sanity check
        text = ' '.join(self.text[i].split())

        # tokenize using Huggingface tokenizers
        out = self.tokenizer.encode_plus(text, None, 
                                   add_special_tokens=True, 
                                   max_length = self.max_len)
        
        ids = out['input_ids']
        mask = out['attention_mask']
        
        padding_length = self.max_len - len(ids)
        ids = ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.label[i], dtype=torch.float)
        }

## Utils

In [11]:
class AverageMeter():
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Engine

In [14]:
def loss_fn(y_pred, y_true):
        return nn.CrossEntropyLoss()(y_pred, y_true[..., None])

def train(train_dl, model, optimizer, device, scheduler=None):
    model.train()

    loss_all = AverageMeter()
    all_losses = []
    losses = 0.
    p_bar = notebook.tqdm(train_dl, total=len(train_dl))
    for i, batch in enumerate(p_bar):
        ids = batch['ids']
        mask = batch['mask']
        targets = batch['targets']

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        model.zero_grad()

        out = model(ids, mask)

        loss = loss_fn(out, targets)
        all_losses.append(loss.item())
        losses += loss.cpu().detach().item()
        loss.backward()
        xm.optimizer_step(optimizer)
        scheduler.step()  #supposed to be after epoch

        # calc metrics
        
        break
    # accuracy_all.update(np.mean(accuracys), ids.size(0))
    # auc_all.update(np.mean(aucs), ids.size(0))
    loss_all.update(loss.item(), ids.size(0))
    p_bar.set_postfix(loss=loss_all.avg)

def evaluate(valid_dl, model, device):
    model.eval()
    all_losses = []
    fin_targs = []
    fin_outs = []
    losses = 0.
    loss_all = AverageMeter()

    with torch.no_grad():
        p_bar = notebook.tqdm(valid_dl, total=len(valid_dl))
        for i, batch in enumerate(p_bar):
            ids = batch['ids']
            mask = batch['mask']
            targets = batch['targets']

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            # model.zero_grad()
            out = model(ids, mask)
            loss = loss_fn(out, targets)

            all_losses.append(loss.item())
            targ_np = targets.cpu().detach().numpy().tolist()

            soft_out = F.softmax(out.cpu().detach(), dim=-1)
            out_np = soft_out.argmax(-1).numpy().tolist()

            fin_targs.extend(targ_np)
            fin_outs.extend(out_np)
            break

        loss_all.update(loss.item(), ids.size(0))
        p_bar.set_postfix(loss=loss_all.avg)
    return fin_targs, fin_outs, losses

## MODEL

In [31]:
class ClasModel(nn.Module):
    def __init__(self, pretrained=True, num_classes=27):
        super(ClasModel, self).__init__()
        if pretrained:
            self.model = config.CLAS_MODEL.from_pretrained(config.MODEL_NAME, config=config.MODEL_CONFIG)
        else: 
            self.model = config.CLAS_MODEL(config.MODEL_CONFIG)
                  
        self.drop = nn.Dropout(0.4)

        self.lin = nn.Linear(768*2, num_classes)
    
    def forward(self, ids, mask):

        h_0, _ = self.model(ids, attention_mask=mask)
        
        mean_pool = torch.mean(h_0, 1)

        max_pool = torch.max(h_0, 1)[0]

        out = torch.cat([mean_pool, max_pool], 1)

        out = self.lin(self.drop(out))

        return out

    # def load_lm_encoder(self,clas_model, lm_path=None):
    #     clas_model_dict = clas_model.state_dict()
    #     if lm_path is not None:
    #         lm_model_dict = torch.load(lm_path).model.state_dict()
    #         needed_dict = {k[6:]:v for k, v in lm_model_dict.items() if str(k)[6:] in clas_model_dict.keys()}
    #         clas_model_dict.update(needed_dict)
    #     clas_model.load_state_dict(clas_model_dict)

## Prep data

In [49]:
!tar -xzf $colab_path/'dataset/temp.tgz'

In [35]:
df_all = pd.read_csv(config.BASE_PATH/'dataset/df_all.csv'); 
df_all.fillna(' ', inplace=True)
df_all.sample(1)

,Title,Description,Image_id,Product_id,Prdtypecode,Prdlbl,image_path
2152,"Lot 4 Livres Partitions Collection ""A Tempo"" (...",,1253468564,3856816679,2403,2403,/content/SIGIR-2020-EComDC-release/image/image...


In [36]:
le = LabelEncoder()
le.fit(df_all.Prdlbl)
le.classes_

array([  10,   40,   50,   60, 1140, 1160, 1180, 1280, 1281, 1300, 1301,
       1302, 1320, 1560, 1920, 1940, 2060, 2220, 2280, 2403, 2462, 2522,
       2582, 2583, 2585, 2705, 2905])

In [37]:
df_all.Prdlbl = le.transform(df_all.Prdlbl)

In [38]:
train_df, valid_df = train_test_split(df_all, test_size=0.2, shuffle=True, stratify=df_all.Prdlbl.values)
train_df.reset_index(drop=True, inplace=True), valid_df.reset_index(drop=True, inplace=True)

(None, None)

##Train

In [43]:
model = ClasModel(pretrained=True)
def world_func():
    xtra_config = config.MODEL_CONFIG
    device = xm.xla_device()

    global model
    # print('before model on device')
    model.to(device)
    # print('after model on device')

    def loss_fn(y_pred, y_true):
        # print(y_pred)
        # print(y_true)
        return nn.CrossEntropyLoss()(y_pred, y_true)

    def train(train_dl, model, optimizer, device, scheduler=None):
        model.train()
        # accuracy_all = AverageMeter()
        # auc_all = AverageMeter()
        loss_all = AverageMeter()
        all_losses = []
        losses = 0.
        # p_bar = notebook.tqdm(train_dl, total=len(train_dl))
        for i, batch in enumerate(train_dl):
            ids = batch['ids']
            mask = batch['mask']
            targets = batch['targets']

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            optimizer.zero_grad()
            model.zero_grad()

            # print('before model in train')
            out = model(ids, mask)

            # print('after model in train')

            loss = loss_fn(out, targets)
            all_losses.append(loss.item())
            losses += loss.cpu().detach().item()
            # print('after loss')

            # if i % 100 == 0: xm.master_print(f'index: {i}, loss: {loss}')

            loss.backward()
            xm.optimizer_step(optimizer)

            if scheduler is not None:
                scheduler.step()  #supposed to be after epoch
            
            # print('after batch')

            # calc metrics
            
            # break
        # accuracy_all.update(np.mean(accuracys), ids.size(0))
        # auc_all.update(np.mean(aucs), ids.size(0))
        loss_all.update(loss.item(), ids.size(0))
        # p_bar.set_postfix(loss=loss_all.avg)

    def evaluate(valid_dl, model, device):
        model.eval()
        all_losses = []
        fin_targs = []
        fin_outs = []
        losses = 0.
        loss_all = AverageMeter()

        with torch.no_grad():
            # p_bar = notebook.tqdm(valid_dl, total=len(valid_dl))
            for i, batch in enumerate(valid_dl):
                ids = batch['ids']
                mask = batch['mask']
                targets = batch['targets']

                ids = ids.to(device, dtype=torch.long)
                mask = mask.to(device, dtype=torch.long)
                targets = targets.to(device, dtype=torch.float)

                # model.zero_grad()
                out = model(ids, mask)
                loss = loss_fn(out, targets)

                all_losses.append(loss.item())
                targ_np = targets.cpu().detach().numpy().tolist()

                soft_out = nn.Softmax(dim=1)(out.cpu().detach())
                out_np = soft_out.argmax(-1).numpy().tolist()

                fin_targs.extend(targ_np)
                fin_outs.extend(out_np)
                # break

            loss_all.update(loss.item(), ids.size(0))
            # p_bar.set_postfix(loss=loss_all.avg)
        return fin_targs, fin_outs, losses


    train_ds = BertDataset((train_df.Title + 'xxfld' + train_df.Description).values, train_df.Prdlbl.values)
    valid_ds = BertDataset((valid_df.Title + 'xxfld' + valid_df.Description).values, valid_df.Prdlbl.values)

    ###########change happens here#####################
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_ds, 
                                                                    num_replicas=xm.xrt_world_size(),
                                                                    rank=xm.get_ordinal(),
                                                                    shuffle=True)
    ###################################################

    train_dl = torch.utils.data.DataLoader(train_ds, batch_size=config.TRAIN_BATCH_SIZE, 
                                        drop_last=True, num_workers=4, 
                                        sampler=train_sampler)

    ###########change happens here#####################
    valid_sampler = torch.utils.data.distributed.DistributedSampler(valid_ds, 
                                                                    num_replicas=xm.xrt_world_size(),
                                                                    rank=xm.get_ordinal(),
                                                                    shuffle=False)
    ###################################################

    valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=config.VALID_BATCH_SIZE, 
                                        drop_last=False, num_workers=4,
                                        sampler=valid_sampler)


    ############change happens here################
    lr = 1e-05 * xm.xrt_world_size()
    #############################

    model_params = list(model.named_parameters())
    # print('after model')

    # we don't want weight decay for these
    no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']

    optimizer_params = [
        {'params': [p for n, p in model_params if n not in no_decay], 
        'weight_decay':0.001},
        #  no weight decay should be applied
        {'params': [p for n, p in model_params if n in no_decay],
        'weight_decay':0.0}
    ]

    optimizer = AdamW(optimizer_params, lr=lr)

    ###############change happens here#########################
    # scheduler
    num_train_steps = int(len(train_df)/ config.TRAIN_BATCH_SIZE / xm.xrt_world_size() * config.NUM_EPOCHS)
    ###########################################################

    xm.master_print(f'num_train_steps = {num_train_steps}, world_size = {xm.xrt_world_size()}')

    scheduler = get_linear_schedule_with_warmup(optimizer=optimizer, 
                                                num_warmup_steps=0, 
                                                num_training_steps=num_train_steps)
    # print('before epoch')
    best_accuracy = 0.
    for epoch in range(config.NUM_EPOCHS):
        train_para_loader = pl.ParallelLoader(train_dl, [device])
        valid_para_loader = pl.ParallelLoader(train_dl, [device])

        start = time.time()
        # print('before train')
        train(train_para_loader.per_device_loader(device), model, optimizer, device, scheduler)
        end = time.time()

        xm.master_print(f'Training time: {end-start} secs')

        fin_targs, fin_outs, losses = evaluate(valid_para_loader.per_device_loader(device), model, device)

        # calc metrics
        accuracy = accuracy_score(fin_targs, fin_outs)
        macro_f1 = f1_score(fin_targs, fin_outs, average='macro')

        # if epoch % 10 == 0:
        xm.master_print(f'VALID ACCURACY: {accuracy}')
        xm.master_print(f'VALID MACRO_F1: {macro_f1}')
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            xm.save(model.state_dict(), config.MODEL_PATH/'torch_xla_pretrained_camembert.bin')

        # break

In [44]:
import gc; gc.collect()

1165

In [45]:
FLAGS = {}
def mp_wrapper(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    world_func()

xmp.spawn(mp_wrapper, args=(FLAGS,), nprocs=8, start_method='fork')

num_train_steps = 796, world_size = 8
Training time: 373.9767234325409 secs
VALID ACCURACY: 0.8299528301886793
VALID MACRO_F1: 0.7666145883370509
Training time: 359.4210033416748 secs
VALID ACCURACY: 0.8854952830188679
VALID MACRO_F1: 0.8554114542066732
Training time: 350.73388600349426 secs
VALID ACCURACY: 0.9058962264150944
VALID MACRO_F1: 0.8849048035940029


If you using a big model, declare the model above multiprocessor func as a global. Also cahnge the pytorch_xla version to nightlyn

## Inference

In [54]:
test_df = pd.read_csv(config.COLAB_PATH/'SIGIR-2020-EComDC-release/data/x_test_task1_phase1.tsv', sep='\t').fillna(' ')
model_path = config.MODEL_PATH/'torch_xla_pretrained_camembert.bin'
test_df['Prdlbl'] = 0
test_df.sample(1)

,Integer_id,Title,Description,Image_id,Product_id,Prdlbl
622,622,La Dame De Pique,,1135788756,2313282280,0


In [55]:
test_ds = BertDataset((test_df.Title + 'xxfld' + test_df.Description).values, test_df.Prdlbl.values)

test_dl = torch.utils.data.DataLoader(test_ds, batch_size=config.VALID_BATCH_SIZE, 
                                        num_workers=4, shuffle=False)

In [62]:
device = xm.xla_device()
model = ClasModel(pretrained=True).to(device)
model.load_state_dict(torch.load(str(model_path)))

<All keys matched successfully>

In [59]:
from tqdm.notebook import tqdm

In [63]:
model.eval()

fin_outs = []

with torch.no_grad():
    for bi, batch in tqdm(enumerate(test_dl), total=len(test_dl)):
        ids = batch['ids']
        mask = batch['mask']
        targets = batch['targets']

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        # model.zero_grad()
        out = model(ids, mask)

        soft_out = nn.Softmax(dim=1)(out.cpu().detach())
        out_np = soft_out.argmax(-1).numpy().tolist()

        fin_outs.extend(out_np)

In [70]:
sub_df = test_df.copy()
sub_df['Prdtypecode'] = le.inverse_transform(fin_outs)
sub_df.head()

,Integer_id,Title,Description,Image_id,Product_id,Prdlbl,Prdtypecode
0,0,Jeep Police - Gevarm-Gevarm,,1193217616,3136702026,0,1300
1,1,Court Joyeux Noël En Peluche Taie Sofa Set Pad...,Joyeux Noël en peluche court Taie Sofa Set Pad...,1323615566,4231863665,0,1920
2,2,Sauna infrarouge Largo - 170 x 105 x 190 - Pin...,Dimensions : 150x105x190 cm ou 170x105x190 cm ...,1158121321,2695198357,0,2583
3,3,BAGUE POUR LAME SOUS-SOLEUSE G. ET D.,,1096607258,1657064583,0,2585
4,4,Carnet De Notes Bloc-Notes Cahierindian Squele...,Taille: En format A5 (144 cm x 21 cm) Caract¿¿...,1303625028,4159071068,0,2522


In [72]:
sub_df[['Integer_id', 'Image_id', 'Product_id', 'Prdtypecode']].to_csv(config.DATA_PATH/'y_test_task1_phase1_pred.tsv', index=False, sep='\t')

In [73]:
sub_df[['Integer_id', 'Image_id', 'Product_id', 'Prdtypecode']].to_csv(config.COLAB_PATH/'y_test_task1_phase1_pred.tsv', index=False, sep='\t')

In [74]:
sub_df[['Integer_id', 'Image_id', 'Product_id', 'Prdtypecode']]

,Integer_id,Image_id,Product_id,Prdtypecode
0,0,1193217616,3136702026,1300
1,1,1323615566,4231863665,1920
2,2,1158121321,2695198357,2583
3,3,1096607258,1657064583,2585
4,4,1303625028,4159071068,2522
...,...,...,...,...
932,932,450840330,50292005,2280
933,933,1275609312,4006500599,1281
934,934,1260304262,3893329037,2583
935,935,931361182,185935391,2522
